In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix,roc_auc_score,ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from tensorflow import keras
from time import asctime

In [2]:
scaler=StandardScaler()

In [3]:
logistic=MLPClassifier(hidden_layer_sizes=(512,512,512,512,512),max_iter=10,tol=1e-15,solver="adam",verbose=True,warm_start=True,n_iter_no_change=1000,learning_rate="adaptive",early_stopping=True,validation_fraction=0.20)

print("reading file")

reading file


In [4]:
train=pd.read_csv("train_transaction.csv")

In [ ]:
train.head(10)

In [5]:
print("file read.")
df=train.copy()
categoricals=df.select_dtypes("object").columns.tolist()

file read.


In [6]:
print("fixing categoricals")
for obj_col in categoricals:
    temp=pd.get_dummies(df[obj_col])
    temp.columns=[f"{obj_col}_{col}" for col in temp.columns.tolist()]
    df=pd.concat([df.drop(columns=[obj_col]),temp],axis=1)
df.drop(columns=["TransactionID","TransactionDT"],inplace=True)
Xy=pd.DataFrame(df.values,columns=df.columns).drop(columns=["isFraud"])


fixing categoricals


In [7]:
Xy.head()

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,...,M5_F,M5_T,M6_F,M6_T,M7_F,M7_T,M8_F,M8_T,M9_F,M9_T
0,68.5,13926.0,NaN,150.0,142.0,315.0,87.0,19.0,NaN,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29.0,2755.0,404.0,150.0,102.0,325.0,87.0,NaN,NaN,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,59.0,4663.0,490.0,150.0,166.0,330.0,87.0,287.0,NaN,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,50.0,18132.0,567.0,150.0,117.0,476.0,87.0,NaN,NaN,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50.0,4497.0,514.0,150.0,102.0,420.0,87.0,NaN,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 
Xy["fraud"]=train.isFraud.copy()
print("creating validation set")
validate=Xy.sample(frac=0.20) 
Xy=Xy[Xy.index.isin(validate.index)==False]


creating validation set


In [9]:
ones=ones=1/Xy.fraud.sum()
zeros=1/len(np.where(Xy.fraud==0)[0])
class_weights={0:zeros,1:ones}
rf=RandomForestClassifier(class_weight=class_weights)
sample=Xy.sample(frac=1)#was 0.20
sample_X=scaler.fit_transform(sample.drop(columns="fraud").fillna(0).values)
sample_X_df=pd.DataFrame(sample_X,columns=Xy.columns[:-1])

In [10]:
print("reducing float64s to round 8")
sample_X=np.round(sample_X,8)        
sample_y=sample.fraud

reducing float64s to round 8


In [11]:
print("getting correlations")
corrs=pd.DataFrame(Xy.drop(columns="fraud"),columns=Xy.columns[:-1]).corr()
print(asctime())

getting correlations
Thu Feb 17 21:17:31 2022


In [12]:
print("getting importances")
corrs_stack=corrs.index.tolist()
importances=pd.DataFrame(rf.fit(sample_X_df[corrs_stack],sample_y).feature_importances_.reshape(1,-1),columns=corrs_stack).T.reset_index().values.tolist()
print(asctime())

getting importances
Thu Feb 17 21:22:02 2022


In [13]:
corrs_stack=[i[0] for i in sorted(importances,key=lambda x:x[1],reverse=True) if i[1]!=0]
print("getting highly correlated fields")

for i in corrs_stack:
    test=corrs.loc[i].reset_index()
    test=test[abs(test[i])>0.80]
    if len(test[test["index"]!=i])>0:
        for r in test[test["index"]!=i]["index"].unique().tolist():
            try:
                corrs_stack.remove(r)
            except:
                print(len(corrs_stack))


getting highly correlated fields
524
519
447
447
447
447
447
447
446
446
446
446
446
446
446
446
446
446
446
437
437
437
437
437
437
437
437
437
437
437
437
435
435
434
429
429
428
396
396
396
396
396
396
396
396
396
396
396
396
396
396
396
392
392
392
392
392
391
391
391
391
390
390
390
390
390
390
390
390
389
388
388
388
388
388
386
385
385
385
378
378
378
377
377
360
360
356
356
355
353
353
353
353
353
353
348
347
346
333
331
331
331
328
328
328
328
328
325
324
324
316
316
316
316
316
315
315
315
314
312
312
312
312
312
312
312
312
312
312
312
312
312
312
310
310
309
307
307
307
307
307
307
306
306
306
306
306
304
304
304
304
304
304
304
302
302
299
299
299
299
299
299
299
299
299
297
297
296
296
295
288


In [14]:
df=df[corrs_stack]
Xy=Xy[corrs_stack+["fraud"]]


In [15]:
auc_stack=[]
val_auc_stack=[]

scaler.fit(Xy[corrs_stack])

StandardScaler()

In [16]:
X=pd.DataFrame(scaler.transform(Xy[corrs_stack].fillna(0)),columns=Xy.columns[:-1])
y=Xy.fraud.reset_index(drop=True)
X_test=pd.DataFrame(scaler.transform(validate[corrs_stack].fillna(0)),columns=corrs_stack)
y_test=validate.fraud
X=np.round(X,8)

In [17]:
logistic=MLPClassifier(hidden_layer_sizes=(256,256,256),max_iter=10,tol=1e-15,solver="adam",learning_rate_init=0.001,verbose=True,warm_start=True,n_iter_no_change=1000,learning_rate="adaptive")

In [18]:
choices=[21,51,61,81,101]
p=[0.20,0.30,0.20,0.25,0.05]
auc_val_stack=[]
print("starting training")
choice=np.random.choice(choices,p=p)
sample_pct=1.0
y_train=y.sample(frac=sample_pct)
X_train=X.loc[y_train.index]

starting training


In [19]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(X_train.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
        
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               69888     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 201,729
Trainable params: 201,729
Non-trai

In [20]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.AUC(name="auc"),
]


In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)
callbacks = [keras.callbacks.EarlyStopping(monitor='val_auc', patience=200),keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}_val_scores_tp_{val_tp}_fn_{val_fn}_tn_{val_tn}_fp_{val_fp}_auc_{val_auc}.h5")]
class_weight = class_weights

In [22]:
model.fit(
    X_train.values,
    y_train.values,
    batch_size=2048,
    epochs=200,
    verbose=0,
    callbacks=callbacks,
    validation_data=(X_test.values, y_test.values),
    class_weight=class_weight,
    
)


In [60]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[5]*100)

Test loss: 0.3060910999774933
Test accuracy: 91.24916791915894


In [73]:
print(score)

[0.3060910999774933, 821.0, 14264.0, 99785.0, 3238.0, 0.9124916791915894]


In [74]:
y_pred = model.predict(X_test)
X_test["pred_prob"]=y_pred
X_test["preds"]=np.where(y_pred > 0.5, 1, 0)

ValueError: in user code:

    File "D:\Anaconda\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "D:\Anaconda\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\Anaconda\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "D:\Anaconda\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "D:\Anaconda\envs\mlenv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\Anaconda\envs\mlenv\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{

In [72]:
X_test.head(10)

,TransactionAmt,card1,card2,C13,addr1,card5,C1,D15,V258,D1,V91,V317,dist1,D10,D4,V283,D3,card6_credit,D11,V310,D8,C5,V217,D5,V312,V201,V315,P_emaildomain_gmail.com,V264,ProductCD_C,V257,V171,card4_mastercard,card3,V263,D9,V282,V285,M6_F,P_emaildomain_yahoo.com,dist2,ProductCD_W,M4_M0,M6_T,R_emaildomain_gmail.com,V45,M5_T,P_emaildomain_hotmail.com,V83,V291,V62,M5_F,V309,M8_F,P_emaildomain_anonymous.com,V87,M9_T,V76,V54,M4_M1,V38,M3_T,V20,M7_F,V281,V320,V13,M8_T,V78,M3_F,V36,R_emaildomain_anonymous.com,V5,R_emaildomain_hotmail.com,V56,M9_F,V190,V289,P_emaildomain_aol.com,V222,V170,ProductCD_R,V47,V244,V7,V3,M2_F,V67,V77,V198,V100,V46,M7_T,V55,V24,card4_discover,D14,V284,ProductCD_H,V6,V2,V9,V140,V137,V261,V290,V158,V124,V160,V188,V26,V319,V41,V303,M1_T,R_emaildomain_yahoo.com,V187,V98,V286,V260,D13,V267,V1,V301,P_emaildomain_icloud.com,V125,V210,V251,V209,P_emaildomain_comcast.net,V234,V256,P_emaildomain_outlook.com,V270,card4_american express,V40,V208,V146,V115,V297,V277,V166,V205,P_emaildomain_live.com,V228,P_emaildomain_mail.com,V116,ProductCD_S,R_emaildomain_icloud.com,P_emaildomain_ymail.com,V197,V109,P_emaildomain_msn.com,R_emaildomain_outlook.com,V247,V239,R_emaildomain_aol.com,V227,V226,V172,P_emaildomain_yahoo.com.mx,V175,V173,V241,P_emaildomain_att.net,V169,V184,V174,P_emaildomain_verizon.net,P_emaildomain_bellsouth.net,P_emaildomain_cox.net,P_emaildomain_optonline.net,V339,V162,P_emaildomain_charter.net,C3,V220,P_emaildomain_me.com,R_emaildomain_mail.com,P_emaildomain_sbcglobal.net,V110,P_emaildomain_protonmail.com,V112,P_emaildomain_embarqmail.com,V240,R_emaildomain_comcast.net,R_emaildomain_outlook.es,R_emaildomain_cox.net,V121,V141,P_emaildomain_live.com.mx,V122,P_emaildomain_roadrunner.com,R_emaildomain_protonmail.com,R_emaildomain_live.com,P_emaildomain_aim.com,P_emaildomain_earthlink.net,V138,V111,P_emaildomain_gmail,P_emaildomain_mac.com,R_emaildomain_me.com,R_emaildomain_msn.com,V118,P_emaildomain_frontier.com,R_emaildomain_prodigy.net.mx,P_emaildomain_cableone.net,R_emaildomain_hotmail.es,R_emaildomain_sbcglobal.net,V120,V119,R_emaildomain_aim.com,R_emaildomain_verizon.net,V107,R_emaildomain_netzero.net,R_emaildomain_charter.net,R_emaildomain_optonline.net,R_emaildomain_rocketmail.com,R_emaildomain_att.net,R_emaildomain_earthlink.net,P_emaildomain_rocketmail.com,P_emaildomain_juno.com,R_emaildomain_bellsouth.net,P_emaildomain_web.de,R_emaildomain_ymail.com,P_emaildomain_hotmail.fr,R_emaildomain_mac.com,P_emaildomain_frontiernet.net,P_emaildomain_sc.rr.com,P_emaildomain_twc.com,P_emaildomain_windstream.net,R_emaildomain_gmx.de,R_emaildomain_yahoo.es,R_emaildomain_yahoo.fr,P_emaildomain_centurylink.net,P_emaildomain_suddenlink.net,P_emaildomain_hotmail.co.uk,P_emaildomain_netzero.com,P_emaildomain_yahoo.es,P_emaildomain_netzero.net,V28,R_emaildomain_yahoo.de,P_emaildomain_cfl.rr.com,R_emaildomain_roadrunner.com,R_emaildomain_embarqmail.com,R_emaildomain_yahoo.co.uk,P_emaildomain_q.com,R_emaildomain_gmail,P_emaildomain_ptd.net,R_emaildomain_frontier.com,R_emaildomain_hotmail.de,R_emaildomain_twc.com,P_emaildomain_yahoo.co.uk,R_emaildomain_juno.com,R_emaildomain_yahoo.co.jp,R_emaildomain_cableone.net,R_emaildomain_live.fr,R_emaildomain_windstream.net,R_emaildomain_netzero.com,R_emaildomain_cfl.rr.com,R_emaildomain_suddenlink.net,R_emaildomain_q.com,P_emaildomain_servicios-ta.com,R_emaildomain_centurylink.net,card6_charge card,R_emaildomain_ptd.net,V305,card6_debit or credit,R_emaildomain_frontiernet.net,M1_F,R_emaildomain_sc.rr.com,R_emaildomain_scranton.edu,pred_prob,preds
0,-0.145892,-1.464696,0.230479,-0.243704,0.945895,0.891215,-0.098210,-0.807602,-0.171034,-0.598186,-0.747033,-0.062156,-0.317881,-0.678885,-0.732358,0.005708,-0.453796,1.719915,-0.788852,-0.333061,-0.631119,-0.215756,-0.110634,-0.475559,-0.222446,-0.113211,-0.228897,1.260137,-0.087737,-0.362753,-0.192913,-0.284773,-0.687933,-0.281926,-0.088408,-1.768309,0.197599,-0.356324,-0.791830,-0.453834,-0.443129,-1.705366,-0.706119,-0.697861,3.052657,-1.